In [1]:
using BenchmarkTools
using ForneyLab

┌ Info: Precompiling ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9]
└ @ Base loading.jl:1273


In [10]:
# Data
    n_samples = 100
    x = [ t for t = 1:n_samples ] # True state
    y = x + sqrt(200.0) * randn(n_samples); # Noisy observations of state 
    
    g = FactorGraph()

    # declare priors as random variables
    @RV m_x_t_min # m_x_t_min = Variable(id=:m_x_t_min)
    @RV v_x_t_min # v_x_t_min = Variable(id=:v_x_t_min)
    @RV x_t_min ~ GaussianMeanVariance(m_x_t_min, v_x_t_min)

    # System equations
    # u = 1.0; v = 200.0
    @RV n_t ~ GaussianMeanVariance(0.0, 200.0)
    @RV x_t = x_t_min + 1.0
    @RV y_t = x_t + n_t

    # Name variable for ease of lookup
    #x_t.id = :x_t;
    
    # Placeholders for prior
    placeholder(m_x_t_min, :m_x_t_min) # placeholder(:m_x_t_min) does not work
    placeholder(v_x_t_min, :v_x_t_min)

    # Placeholder for data
    placeholder(y_t, :y_t);
    
    algo = sumProductAlgorithm(x_t) # Figure out a schedule and compile to Julia code
    
    # Define algorithm
    eval(Meta.parse(algo))
    
    # Define values for prior statistics
    m_x_0 = 0.0
    v_x_0 = 1000.0

    m_x = Vector{Float64}(undef, n_samples)
    v_x = Vector{Float64}(undef, n_samples)

    m_x_t_min = m_x_0
    v_x_t_min = v_x_0
    for t = 1:n_samples
        # Prepare data and prior statistics
        data = Dict(:y_t       => y[t],
                    :m_x_t_min => m_x_t_min,
                    :v_x_t_min => v_x_t_min)

        # Execute algorithm
        marginals = step!(data)

        # Extract posterior statistics
        m_x_t_min = mean(marginals[:x_t])
        v_x_t_min = var(marginals[:x_t])

        # Store to buffer 
        m_x[t] = m_x_t_min
        v_x[t] = v_x_t_min
    end
    
    m_x, v_x

([12.981357574851947, 15.69586056695893, 10.910347523280167, 14.278805961913577, 16.15408249871205, 15.23918905676211, 11.767319245441332, 14.68066644382486, 13.078509977978948, 14.162016702480793  …  93.29325153998214, 94.4442106712839, 95.32442824079817, 96.39767938518047, 97.34156654214516, 97.92259671713069, 98.79782680527563, 99.70045198514951, 100.62204780657096, 101.49497049797917], [166.66666666666666, 90.90909090909092, 62.5, 47.61904761904761, 38.46153846153846, 32.25806451612903, 27.77777777777778, 24.39024390243903, 21.739130434782613, 19.607843137254907  …  2.1929824561403493, 2.169197396963122, 2.1459227467811144, 2.123142250530784, 2.1008403361344525, 2.079002079002078, 2.057613168724279, 2.0366598778004064, 2.0161290322580636, 1.9960079840319351])

In [11]:
m_x

100-element Array{Float64,1}:
  12.981357574851947
  15.69586056695893 
  10.910347523280167
  14.278805961913577
  16.15408249871205 
  15.23918905676211 
  11.767319245441332
  14.68066644382486 
  13.078509977978948
  14.162016702480793
  15.647581381633776
  16.646892074441165
  16.62560553773851 
   ⋮                
  90.88620487789015 
  92.13596946463926 
  93.29325153998214 
  94.4442106712839  
  95.32442824079817 
  96.39767938518047 
  97.34156654214516 
  97.92259671713069 
  98.79782680527563 
  99.70045198514951 
 100.62204780657096 
 101.49497049797917 

In [12]:
v_x

100-element Array{Float64,1}:
 166.66666666666666  
  90.90909090909092  
  62.5               
  47.61904761904761  
  38.46153846153846  
  32.25806451612903  
  27.77777777777778  
  24.39024390243903  
  21.739130434782613 
  19.607843137254907 
  17.85714285714286  
  16.393442622950825 
  15.151515151515158 
   ⋮                 
   2.242152466367712 
   2.217294900221728 
   2.1929824561403493
   2.169197396963122 
   2.1459227467811144
   2.123142250530784 
   2.1008403361344525
   2.079002079002078 
   2.057613168724279 
   2.0366598778004064
   2.0161290322580636
   1.9960079840319351